# AM2 Parameter Estimation with Optuna

This notebook demonstrates how to calibrate the AM2 model parameters to fit experimental data using Bayesian optimization via **Optuna**.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from openad_lib.models.mechanistic import AM2Model
from openad_lib.optimisation import AM2Calibrator

%matplotlib inline

## 1. Setup Model and Data

In [ ]:
# Load sample data
import openad_lib
data_path = os.path.join(os.path.dirname(openad_lib.__file__), 'data', 'sample_AM2_data.csv')
print(f"Loading data from: {data_path}")

model = AM2Model()
model.load_data(data_path)

# Check initial performance
print("Initial Performance:")
model.run(verbose=False)
model.print_metrics()

## 2. Configure Calibration
We initialize the `AM2Calibrator` and define which parameters to tune.
We'll focus on the kinetic parameters: growth rates ($m_1, m_2$) and saturation constants ($K_1, K_i, K_2$).

In [ ]:
calibrator = AM2Calibrator(model)

# Parameters to optimize
params_to_tune = ['m1', 'K1', 'm2', 'Ki', 'K2']

# Weights for the objective function
# We give equal weight to VFA (S2) and Biogas (Q), and slightly less to COD (S1)
weights = {'S1': 0.5, 'S2': 1.0, 'Q': 1.0}

## 3. Run Optimization
We'll run 50 trials using Optuna.

In [ ]:
best_params = calibrator.calibrate(
    params_to_tune=params_to_tune,
    n_trials=50,
    weights=weights,
    show_progress_bar=True
)

## 4. Visualization and Analysis
Let's visualize the optimization history and parameter importance.

In [ ]:
# Optimization History
calibrator.plot_optimization_history()

In [ ]:
# Parameter Importance
calibrator.plot_param_importances()

## 5. Verify Calibrated Model
Run the simulation with the new best parameters and compare fits.

In [ ]:
print("Calibrated Performance:")
model.run(verbose=False)
model.print_metrics()

model.plot_results(figsize=(12, 10), show_measured=True)